In [42]:
pathtorepo = "C:\\Users\\Alex\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

using StaticArrays, DifferentialEquations, DynamicalSystems

x, y, z  = -10..10, -10..10,  -10..10
box = x × y × z

using CairoMakie
using GLMakie
using ForwardDiff
using LinearAlgebra

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [88]:
@inbounds function Res(u, p, t)
    du1 = -u[2]-u[3]
    du2 = u[1]+p[1]*u[2]
    du3 = p[2]*u[1]-p[3]*u[3]+u[1]*u[3]
    return SVector(du1, du2, du3)
end
@inbounds function jac_res(u, p, t)
    SMatrix{3, 3}(0.0, 1.0, p[2]+u[3],
                -1.0, p[1], 0.0,
                -1.0, 0.0, -p[3]+u[1])
end

jac_res (generic function with 1 method)

In [216]:
time_set = (t = 500, Ttr = 250, tstep = 0.001)
integ_set = (alg = Vern9(), adaptive = false, dt = time_set.tstep)

(alg = Vern9(stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false), lazy = true), adaptive = false, dt = 0.001)

In [250]:
 b = 0.3;  c = 4.93; a = 0.33;

In [251]:
u0 = SA[0.13060105886751308, 0.4782615699018184, 0.7435866656901606]
p = [a, b, c]
ds = CoupledODEs(Res, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  Res
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [0.33, 0.3, 4.93]
 time:          0.0
 state:         [0.13060105886751308, 0.4782615699018184, 0.7435866656901606]


In [252]:
fp, ei, _ = fixedpoints(ds, box, jac_res)

(3-dimensional StateSpaceSet{Float64} with 1 points, Vector{ComplexF64}[[-4.870746908710997 + 0.0im, 0.13537345435549808 - 0.9866679768306059im, 0.13537345435549808 + 0.9866679768306059im]], Bool[0])

In [272]:
Jacobian = jac_res(fp[1], p, 0)
eivecs = eigvecs(Jacobian)
vec_stable = real(eivecs[:, 1])
ϵ = 1e-19
shift = fp[1] + vec_stable * ϵ

3-element SVector{3, Float64} with indices SOneTo(3):
  4.022650790251932e-20
 -6.694510549923717e-20
  1.612586767974696e-19

In [279]:
prob = ODEProblem(Res, big.(shift), (0.0, -500.0), big.(p))

ODEProblem with uType SVector{3, BigFloat} and tType Float64. In-place: false
timespan: (0.0, -500.0)
u0: 3-element SVector{3, BigFloat} with indices SOneTo(3):
  4.02265079025193171889387999980240252773811803949928462371607440495324681251077e-20
 -6.694510549923716591631731739035637855778818928173046817059477220368535199668258e-20
  1.612586767974695995913035605470240735769538745589702245092356136524358589667827e-19

In [281]:
sol = solve(prob,alg = RK4(), adaptive = false, dt = 0.001)

┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\UGmh1\src\integrator_interface.jl:596


retcode: Unstable
Interpolation: 3rd order Hermite
t: 15122-element Vector{Float64}:
   0.0
  -0.001
  -0.002
  -0.003
  -0.004
  -0.005
  -0.006
  -0.007
  -0.008
  -0.009000000000000001
  -0.010000000000000002
  -0.011000000000000003
  -0.012000000000000004
   ⋮
 -15.109999999997065
 -15.110999999997064
 -15.111999999997064
 -15.112999999997063
 -15.113999999997063
 -15.114999999997062
 -15.115999999997062
 -15.116999999997061
 -15.11799999999706
 -15.11899999999706
 -15.11999999999706
 -15.120999999997059
u: 15122-element Vector{SVector{3, BigFloat}}:
 [4.02265079025193171889387999980240252773811803949928462371607440495324681251077e-20, -6.694510549923716591631731739035637855778818928173046817059477220368535199668258e-20, 1.612586767974695995913035605470240735769538745589702245092356136524358589667827e-19]
 [4.032120449979128422105918217324316108847483290592168033574425142841139738543775e-20, -6.696328440965264427857464616989191875863921053584915606328303322147254367668032e-20, 1.62

In [278]:
ts, tf = 1, 15000
idx, idy, idz = 1, 2, 3
f = Figure(resolution = (1200, 800))
ax = LScene(f[1, 1])
lines!(ax, sol[idx, ts:tf], sol[idy, ts:tf], sol[idz, ts:tf], linewidth = 1.0, color = :deeppink)
scatter!(fp[1][idx], fp[1][idy], fp[1][idz], color = :black)
display(GLMakie.Screen(), f)

GLMakie.Screen(...)